# Day 30: Model Cards and Documentation - Part 2

In this notebook, we'll explore how to create comprehensive model cards and documentation for your fine-tuned models. Good documentation is essential for responsible sharing and use of machine learning models.

## Overview

1. Understanding model cards
2. Creating a model card template
3. Documenting model performance
4. Implementing a model card for our fine-tuned model

## 1. Setup and Dependencies

In [ ]:
!pip install -q transformers datasets pandas matplotlib seaborn

In [ ]:
import os
import json
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import AutoModelForCausalLM, AutoTokenizer
from datetime import datetime

# Set paths
merged_model_path = "./merged-model"  # Path to our merged model from Part 1
model_card_path = "./model-card"  # Path to save our model card

## 2. Understanding Model Cards

Model cards are structured documentation that provide essential information about machine learning models. They were introduced by Mitchell et al. (2019) to promote transparency and accountability in machine learning.

### Key Components of a Model Card

1. **Model Details**: Basic information about the model architecture, version, and creators
2. **Intended Use**: Primary use cases and out-of-scope uses
3. **Training Data**: Information about the data used to train the model
4. **Performance**: Evaluation metrics and benchmarks
5. **Limitations**: Known limitations and biases
6. **Ethical Considerations**: Potential risks and mitigations
7. **Technical Specifications**: Hardware requirements and inference details

## 3. Creating a Model Card Template

Let's create a template for our model card based on the Hugging Face model card specification.

In [ ]:
# Define a model card template
def create_model_card_template(model_name, model_description, base_model, creators, license_info):
    template = f"""---
language: en
license: {license_info}
library_name: transformers
tags:
- lora
- adapter
- fine-tuned
- text-generation
---

# {model_name}

## Model Description

{model_description}

## Model Details

- **Developed by:** {creators}
- **Base Model:** {base_model}
- **Model type:** Fine-tuned language model with LoRA adapters
- **Language(s):** English
- **License:** {license_info}
- **Last updated:** {datetime.now().strftime('%Y-%m-%d')}

## Intended Use

### Primary Intended Uses

- Text generation for [specific use case]
- [Other intended uses]

### Out-of-Scope Uses

- Production systems without human review
- Critical decision-making without oversight
- [Other out-of-scope uses]

## Training Data

- **Training Dataset:** [Dataset name and description]
- **Preprocessing:** [Description of preprocessing steps]
- **Training-Validation Split:** [Split details]

## Performance and Limitations

### Performance Measures

- [Key metrics and results]

### Limitations

- [Known limitations of the model]
- [Potential biases]

## Ethical Considerations

- [Potential risks and ethical concerns]
- [Mitigation strategies]

## Technical Specifications

- **Hardware Requirements:** [Minimum hardware requirements]
- **Inference Speed:** [Inference benchmarks]
- **Model Size:** [Size of model artifacts]

## How to Use

```python
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained("[model_path]")
tokenizer = AutoTokenizer.from_pretrained("[model_path]")

# Generate text
inputs = tokenizer("Your prompt here", return_tensors="pt")
outputs = model.generate(**inputs, max_length=100)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
```

## Citation and Contact

- **Citation:** [How to cite the model]
- **Contact:** [Contact information]
"""
    return template

## 4. Documenting Model Performance

Let's create some example performance metrics for our model. In a real scenario, you would use actual evaluation results.

In [ ]:
# Example performance metrics
performance_metrics = {
    "perplexity": 15.3,
    "accuracy": 0.87,
    "f1_score": 0.85,
    "inference_time_ms": 120,
    "memory_usage_mb": 512
}

# Create a visualization of the metrics
plt.figure(figsize=(10, 6))

# Plot accuracy and F1 score
plt.subplot(1, 2, 1)
metrics = ["accuracy", "f1_score"]
values = [performance_metrics[m] for m in metrics]
plt.bar(metrics, values, color=["blue", "green"])
plt.ylim(0, 1)
plt.title("Performance Metrics")

# Plot perplexity
plt.subplot(1, 2, 2)
plt.bar(["perplexity"], [performance_metrics["perplexity"]], color="red")
plt.title("Perplexity (lower is better)")

plt.tight_layout()
plt.savefig("performance_metrics.png")
plt.show()

## 5. Implementing a Model Card for Our Fine-tuned Model

Now, let's create a specific model card for our fine-tuned model.

In [ ]:
# Load our model to get details
try:
    model = AutoModelForCausalLM.from_pretrained(merged_model_path)
    tokenizer = AutoTokenizer.from_pretrained(merged_model_path)
    
    # Get model details
    model_name = "GPT-2 with LoRA Fine-tuning"
    base_model = "gpt2"
    model_size = sum(p.numel() for p in model.parameters()) / 1_000_000  # in millions
    
    print(f"Model loaded: {model_name}")
    print(f"Base model: {base_model}")
    print(f"Model size: {model_size:.2f}M parameters")
    
except Exception as e:
    print(f"Error loading model: {e}")
    # Use default values if model loading fails
    model_name = "GPT-2 with LoRA Fine-tuning"
    base_model = "gpt2"
    model_size = 124  # GPT-2 has approximately 124M parameters

In [ ]:
# Create our specific model card
model_description = f"""This is a fine-tuned version of {base_model} using LoRA adapters. The model was trained to improve text generation capabilities for specific topics. The LoRA adapters were merged with the base model for efficient inference.

The model uses a rank-8 LoRA configuration targeting attention modules, which allows for efficient fine-tuning while maintaining most of the base model's general knowledge."""

creators = "Your Name"
license_info = "MIT"

model_card = create_model_card_template(
    model_name=model_name,
    model_description=model_description,
    base_model=base_model,
    creators=creators,
    license_info=license_info
)

In [ ]:
# Add performance metrics to the model card
performance_section = f"""
## Performance Measures

- **Perplexity:** {performance_metrics['perplexity']:.2f}
- **Accuracy:** {performance_metrics['accuracy']:.2f}
- **F1 Score:** {performance_metrics['f1_score']:.2f}
- **Inference Time:** {performance_metrics['inference_time_ms']} ms per generation
- **Memory Usage:** {performance_metrics['memory_usage_mb']} MB

![Performance Metrics](performance_metrics.png)
"""

# Replace the placeholder performance section
model_card = model_card.replace("- [Key metrics and results]", performance_section)

In [ ]:
# Add technical specifications
technical_specs = f"""
- **Model Type:** Transformer-based language model
- **Model Size:** {model_size:.2f}M parameters
- **Hardware Requirements:** GPU with at least 4GB VRAM for inference
- **Inference Speed:** {performance_metrics['inference_time_ms']} ms per generation (average)
- **Model Size on Disk:** Approximately 500MB
"""

# Replace the placeholder technical specifications
model_card = model_card.replace("- **Hardware Requirements:** [Minimum hardware requirements]\n- **Inference Speed:** [Inference benchmarks]\n- **Model Size:** [Size of model artifacts]", technical_specs)

In [ ]:
# Add example usage specific to our model
usage_example = f"""
```python
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained("{model_name.lower().replace(' ', '-')}")
tokenizer = AutoTokenizer.from_pretrained("{model_name.lower().replace(' ', '-')}")

# Generate text
prompt = "The future of artificial intelligence is"
inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(**inputs, max_length=100, do_sample=True, temperature=0.7)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
```
"""

# Replace the placeholder usage example
model_card = model_card.replace("```python\nfrom transformers import AutoModelForCausalLM, AutoTokenizer\n\n# Load model and tokenizer\nmodel = AutoModelForCausalLM.from_pretrained(\"[model_path]\")\ntokenizer = AutoTokenizer.from_pretrained(\"[model_path]\")\n\n# Generate text\ninputs = tokenizer(\"Your prompt here\", return_tensors=\"pt\")\noutputs = model.generate(**inputs, max_length=100)\nprint(tokenizer.decode(outputs[0], skip_special_tokens=True))\n```", usage_example)

In [ ]:
# Save the model card
os.makedirs(model_card_path, exist_ok=True)
with open(f"{model_card_path}/README.md", "w") as f:
    f.write(model_card)

# Copy the performance metrics image to the model card directory
import shutil
shutil.copy("performance_metrics.png", f"{model_card_path}/performance_metrics.png")

print(f"Model card saved to {model_card_path}/README.md")

## 6. Viewing the Model Card

Let's take a look at our model card.

In [ ]:
# Display the model card
with open(f"{model_card_path}/README.md", "r") as f:
    model_card_content = f.read()

print(model_card_content)

## 7. Adding Metadata for Model Registries

Many model registries (like Hugging Face Hub) use metadata files to provide additional information about the model.

In [ ]:
# Create metadata for Hugging Face Hub
metadata = {
    "language": "en",
    "license": license_info,
    "library_name": "transformers",
    "tags": ["lora", "adapter", "fine-tuned", "text-generation"],
    "datasets": ["custom"],
    "metrics": ["perplexity", "accuracy"],
    "model-index": [
        {
            "name": model_name,
            "results": [
                {
                    "task": {
                        "type": "text-generation"
                    },
                    "dataset": {
                        "type": "custom",
                        "name": "Custom Dataset"
                    },
                    "metrics": [
                        {
                            "type": "perplexity",
                            "value": performance_metrics["perplexity"],
                            "name": "Perplexity"
                        },
                        {
                            "type": "accuracy",
                            "value": performance_metrics["accuracy"],
                            "name": "Accuracy"
                        }
                    ]
                }
            ]
        }
    ]
}

# Save metadata
with open(f"{model_card_path}/metadata.json", "w") as f:
    json.dump(metadata, f, indent=2)

print(f"Metadata saved to {model_card_path}/metadata.json")

## 8. Creating a License File

Including a license file is important for clarifying how others can use your model.

In [ ]:
# Create an MIT license file
mit_license = f"""MIT License

Copyright (c) {datetime.now().year} {creators}

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
"""

# Save the license file
with open(f"{model_card_path}/LICENSE", "w") as f:
    f.write(mit_license)

print(f"License file saved to {model_card_path}/LICENSE")

## Conclusion

In this notebook, we've explored how to create comprehensive documentation for fine-tuned models:

1. We created a detailed model card following best practices
2. We added performance metrics and visualizations
3. We included metadata for model registries
4. We added a license file to clarify usage rights

Good documentation is essential for responsible model sharing and use. By providing clear information about your model's capabilities, limitations, and intended uses, you help others use your model appropriately and build upon your work.

In a real-world scenario, you would include more detailed information about:
- The specific dataset used for fine-tuning
- Comprehensive evaluation results across different metrics and datasets
- Known biases and limitations based on thorough testing
- Specific ethical considerations relevant to your model's domain

This documentation, combined with the adapter management techniques from Part 1, provides a solid foundation for packaging and sharing your fine-tuned models.